# We Are The World — Analyzing Population Density vs Densely Populated Countries.

In this project, I will analyze data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/) - a compendium of statistics from countries all around the world. The goal is to query data from the factbook using SQL for use in analyzing densely populated areas vs population density. I will use the information gathered to draw some inferences about the following:
- population size and area
- countries and their growth
- Highest # of people in a country
- Population Density vs Densely Populated

Summary of Results
After completing analysis of the dataset, The conclusion that was drawn was that countries that were considered densely populated did not mean that the country had high population density. All that it meant was that densely populated countries had higher than average population values in regards to their area which were mostly below par. 

For more details, refer to the full analysis below.

# Overview of the Data

I'll begin by doing a quick exploration of the data in the CIA World Factbook. First, I'll load sql into jupyter notebook to start querying from the database. 

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: done

# All requested packages already installed.



In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

I'll locate the name of the table by querying it from the database itself.

In [3]:
%%sql 
SELECT *
  FROM sqlite_master
WHERE type= 'table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Now that I have the table name, I'll get an overview of the data and its values. 

In [4]:
%%sql 
SELECT *
  FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


A quick exploration shows the following columns and their descriptions:

- name - The name of the country.
- area - The total land and sea area of the country.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.

# Summary Statistics
I'm going to calcualte some summary statistics for the data by querying the minimum and maximum values for population and population growth. I want to know if their are outlier countries. 

In [5]:
%%sql 
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Looking at the table generated, the following information sticks out:
- there is a country with a population of 0 
- there is a country that has a population of 7256490011(or more than 7.2 billion). 

# Exploring Outliers
From the table above, I see there are two interests things to note:
- a country with a population of 0
- a population with a population over 7.2 billion

I want to dive a little bit deeper into that, so I'll use some subqueries to determine what country/countries those numbers below to.

In [6]:
%%sql 
SELECT Name, MIN(population)
FROM facts

 * sqlite:///factbook.db
Done.


name,MIN(population)
Antarctica,0


Antarctica looks like the first outlier with a population of 0. 

In [7]:
%%sql 
SELECT Name, MAX(population)
FROM facts

 * sqlite:///factbook.db
Done.


name,MAX(population)
World,7256490011


The second outlier is the World with a population of 7256490011.

Taking a deeper look into the CIA World Factbook, a population of 0 seems to make sense for [Antarctica](https://www.cia.gov/the-world-factbook/countries/antarctica/#people-and-society) as there are no indegenious inhabitants. Though it does have permanent and summer-only research stations that are staffed. The other outlier is [the World](https://www.cia.gov/the-world-factbook/countries/world/#people-and-society) which explains the outrageously high population number as it is the sum of all inhabitants on the planet.

# Summary Statistics Revisited
Now that I know the World is include in our database, I need to exclude it from my query. I'm doing this so I can get accurate look at the data(as the World isn't a country). 

I'm going to recompute my summary statistics again but exclude the World this time. 

In [8]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


After excluding the world, there looks there is a country with a population of 1367485388( or more than 1.3 billion).

# Exploring Average Population and Area
Now that I have an accurate summary statistic, I want to explore density which dependent on the population and the country's area. 

I'll start by taking an average of the population and area while also discarding the World as a row in the data. 

In [9]:
%%sql
SELECT AVG(population), AVG(area)
  FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


The average population is around 32 million and the average area is 555 thousand square kilometers

There seems to a huge disparity in average population in comparison to average land which could suggest that some countries are very densely populated.

# Finding Densely Populated Countries
As noted above, there looks to be a huge disparity in comparison between average population to average area. This suggests there are a good amount of countries that are densely populated. 

I'll write a query that identifies the following:
- above average population values
- below average area values

In [10]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE name <> 'World')
AND area < (SELECT AVG(area) FROM facts WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Glancing over the table, some of the countries are known to be densely populated so I can be confident that my results are successful. 

# Exploring Growth Rate and Greatest Amount of People 
Next I want to explore the country that has the highest growth rate and the country that contains the most amount of people. 

In [11]:
%%sql
Select name, MAX(population) as max_pop
  FROM facts 
WHERE population == (SELECT MAX(population) 
                       FROM facts WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


name,max_pop
China,1367485388


In [12]:
%%sql 
SELECT name, MAX(population_growth) as max_pop_growth
  FROM facts
WHERE population_growth == (SELECT MAX(population_growth) 
                       FROM facts WHERE name <> 'World')

 * sqlite:///factbook.db
Done.


name,max_pop_growth
South Sudan,4.02


The country with the most amount of people is China with 13674485388(or more than 1.3 billion) and the country that has the highest growth rate is South Sudan with a growth rate of 4.02%. According to the CIA World Factbook, [China](https://www.cia.gov/the-world-factbook/countries/china/#people-and-society) is #1 in the world in terms of population with a density of 152 people per km^2 according to the 2021 Nations Population Division estimates which can be found [here](https://www.worldometers.info/world-population/population-by-country/). [South Sudan](https://www.cia.gov/the-world-factbook/countries/south-sudan/#people-and-society) had the highest population growth in the world in 2020 when this data was populated. Further research that can be found [here](https://www.worldbank.org/en/country/southsudan/overview) suggests this could have been the result of a formation of unity government in 2020 and decreasing conflict that allowed some refugees to return. 

# Exploring Countries with the Highest Population Density
I want to know how my query for densely populated countries holds up, so I will perform another query to calculate out density or the population/area ratio. The formula for density is the following:
-  Population(P) / Area(km^2)

In [13]:
%%sql
SELECT id, code, name, (population / area) as density
FROM facts
WHERE name <> 'World'
ORDER BY density DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,density
205,mc,Macau,21168
117,mn,Monaco,15267
156,sn,Singapore,8141
204,hk,Hong Kong,6445
251,gz,Gaza Strip,5191
233,gi,Gibraltar,4876
13,ba,Bahrain,1771
108,mv,Maldives,1319
110,mt,Malta,1310
227,bd,Bermuda,1299


Looking at the table above, The country with the highest density(population per square kilometer) is Macau with a ratio of 21168 people per square kilometer. This is confirmed in the 2021 World Population Review which can be found [here](https://worldpopulationreview.com/country-rankings/countries-by-density). Noticably, none of the countries in this table are found in the previous table concerning densely populated countries. 

# Conclusion

As stated in the observation above, comparing to the list of densely populated countries, there is a noticable disconnect between the two, as the two tables do not share a single country in common. This makes sense as population density and being densely populated do not carry the same meaning. Densely populated countries are commonly known as countries that have above average population values in regards to below average area values. Those countries are just the countries that exceed average population expectations while simultaneously are below par in values concerning area. Population density, on the other hand, is seen as the number of people in a population spread over the country's area. The values itself are not compared to an average or any other standard. Overall, 